This notebook is only for testing. Maybe we can build a working model from this, and then we can start putting code into the "model" python module.

In [ ]:
import tensorflow as tf
import numpy as np
import time
import importlib

In [ ]:
import sys
sys.path.append('../py')
import cavitylearn.data
import cavitylearn.catalonet0

## Loading Data

In [ ]:
dataconfig = cavitylearn.data.DataConfig(classes=["SO4","GOL","EDO","HEM","NAG","PO4","ACT"], num_props=8, boxshape=[30, 30, 30])
with open('/var/cavitylearn/dataset-test/labels.txt', 'rt') as labelfile:
    trainset = cavitylearn.data.DataSet(labelfile, '/var/cavitylearn/dataset-test', dataconfig)

In [ ]:
print('Examples available in dataset: ', trainset.N)

## Building the model

In [ ]:
boxes_placeholder = tf.placeholder(tf.float32, shape=[None, dataconfig.boxshape[0], dataconfig.boxshape[1], dataconfig.boxshape[2], dataconfig.num_props])
labels_placholder = tf.placeholder(tf.float32, shape=[None, dataconfig.num_classes])

In [ ]:
logits = cavitylearn.catalonet0.inference(boxes_placeholder, dataconfig)

In [ ]:
loss = cavitylearn.catalonet0.loss(logits, labels_placholder)

In [ ]:
train_op = cavitylearn.catalonet0.training(loss, 1e-4)

In [ ]:
correct_prediction = tf.equal(tf.argmax(logits,1),  tf.argmax(labels_placholder,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")

In [ ]:
tf.scalar_summary("accuracy", accuracy)

In [ ]:
almost_correct_prediction = tf.nn.in_top_k(logits, tf.argmax(labels_placholder, 1), 3)
almost_accuracy = tf.reduce_mean(tf.cast(almost_correct_prediction, tf.float32), name="almost_accuracy")

In [ ]:
tf.scalar_summary("almost_accuracy", almost_accuracy)

## Training the Model

In [ ]:
summary_op = tf.merge_all_summaries()

In [ ]:
BATCHSIZE = 200
NUM_BATCHES = 110

In [ ]:
trainset.rewind_batches()
with tf.Session() as sess:
    summary_writer = tf.train.SummaryWriter('/var/cavitylearn/dataset-test/logdir', sess.graph)
    
    sess.run(tf.initialize_all_variables())
    
    start_time = time.time()
    for i in range(NUM_BATCHES):
        # get training data
        labels, boxes = trainset.next_batch(BATCHSIZE)
        
        if len(labels) == 0:
            break
            
            
        feed_dict = {
            boxes_placeholder: boxes,
            labels_placholder: labels
        }
        _, loss_val, accuracy_val, almost_accuracy_val = sess.run([train_op, loss, accuracy, almost_accuracy], feed_dict=feed_dict)
        print("i = {:>3}; loss: {:>6.2f}; accuracy: {:.0f}%; almost_accuracy: {:.0f}%".format(i, loss_val, accuracy_val*100, almost_accuracy_val*100))
        
        
        summary_str = sess.run(summary_op, feed_dict=feed_dict)
        summary_writer.add_summary(summary_str, i)
        summary_writer.flush()
    end_time = time.time()

In [ ]:
print("Total time: ", str(end_time - start_time))
print("Time per batch: ", str((end_time - start_time)/NUM_BATCHES))

importlib.reload(cavitylearn.catalonet0)
tf.reset_default_graph()